In [ ]:
import base64
import h5py
import IPython.display as ipd
import os
import pandas as pd
import sys

In [ ]:
sys.path.append('/'.join(os.getcwd().split('/')[:-1]))
from library.searcher import Searcher

In [ ]:
%matplotlib inline

### Example usage

In [ ]:
# Node chosen due to proximity to Washington Square Park (next to philosophy department) and has 2017 data.
searcher = Searcher('sonycnode-b827eb2a1bce.sonyc')

In [ ]:
weather_df = pd.read_csv(
    '../data/weather.csv', 
    usecols=['datetime[utc]', 'precipitation[mm]'], 
    parse_dates=['datetime[utc]']
)
weather_df.sort_values(by=['precipitation[mm]'], ascending=False).head(23)
# First occurence of a lot of rain in 2017 at rank 23.
# This gives us the node index 57337

In [ ]:
# Use the node index to find the audio index.
# Audio index 290874 chosen.
searcher.return_interval(weather_df.loc[57337, 'datetime[utc]'])

In [ ]:
ipd.Audio(searcher.get_audio(290874))

In [ ]:
# save data for 2017-01-01
searcher.save_audio_by_day('2017-01-01')